In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import importlib
import wind_fusion.pangu_pytorch as pg

importlib.reload(pg)

# Example usage of the inference function
result = pg.inference()
print(result)

# Compare original pangu_pytorch input data with zarr data from lsdf

In [ ]:
import xarray as xr



old_data_surface = xr.open_dataset("/hkfs/home/project/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion/pangu_pytorch/data/surface/surface_201907.nc")
old_data_surface = old_data_surface.sel(time="2019-07-01 00:00:00")

old_data_upper = xr.open_dataset("/home/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion/pangu_pytorch/data/upper/upper_20190701.nc")
old_data_upper = old_data_upper.sel(time="2019-07-01 00:00:00")

zarr_data = xr.open_dataset("/lsdf/kit/imk-tro/projects/Gruppe_Quinting/ec.era5/1959-2023_01_10-wb13-6h-1440x721.zarr", engine='zarr')
variable_mapping = {
    "geopotential": "z",
    "specific_humidity": "q",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "mean_sea_level_pressure": "msl",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "2m_temperature": "t2m"
}

surface_variables = ["msl", "u10", "v10", "t2m"]
upper_variables = ["z", "q", "t", "u", "v"]

# Select and rename variables according to the mapping
zarr_data = zarr_data[list(variable_mapping.keys())].rename(variable_mapping)

input_upper_dataset = zarr_data[upper_variables]
input_surface_dataset = zarr_data[surface_variables]

input_surface_dataset = input_surface_dataset.sel(time=old_data_surface.time)
input_upper_dataset = input_upper_dataset.sel(time=old_data_surface.time)


In [ ]:
zarr_data

In [ ]:
old_data_surface
# old_data_upper

In [ ]:
# Compare the values of new vs old data (they should be the same)

for var in old_data_surface.data_vars:
    print(f"Variable: {var}")
    print(f"\t99th Percentile: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).quantile(0.99).values}")
    print(f"\t1st Percentile: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).quantile(0.01).values}")
    print(f"\tMean: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).mean().values}")
    print(f"\tStandard Deviation: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).std().values}")
    print(f"\tMax: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).max().values}")
    print(f"\tMin: {((old_data_surface[var] - input_surface_dataset[var])/old_data_surface[var]).min().values}")
    print("")

Resume: The differences are very small. For t2m & msl theyre irrelevant. For v10 and u10, the 99th percentile of deviations is 0.2% of deviations.

In [ ]:
for var in old_data_upper.data_vars:
    print(f"Variable: {var}")
    print(f"\t99th Percentile: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).quantile(0.99).values}")
    print(f"\t1st Percentile: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).quantile(0.01).values}")
    print(f"\tMean: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).mean().values}")
    print(f"\tStandard Deviation: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).std().values}")
    print(f"\tMax: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).max().values}")
    print(f"\tMin: {((old_data_upper[var] - input_upper_dataset[var])/old_data_upper[var]).min().values}")
    print("")

Resume for upper variables: All deviations seem reasonably small, except for q (specific humidity)

In [ ]:
# Print pairwise values of old_data_upper['q'] and input_upper_dataset['q']
old_q_values = old_data_upper['q'].values
input_q_values = input_upper_dataset['q'].values

for old_val, input_val in zip(old_q_values.flatten(), input_q_values.flatten()):
    print(f"Old q: {old_val}, Input q: {input_val}")

After inspecting q (specific humidity):
- The two datasets are not the same.
- However they are still "similar": They stay in the same range
- Probably the differences are due to the numbers being super small (e-6), and old_data uses double precision while zarr data is single precision